In [1]:
# There is a number of ways to build a new dictionary

a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = dict({'three': 3, 'two': 2, 'one':1})
a == b == c == d == e

True

In [2]:
# Similar to lists and tuples there are dict comprehensions
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
]
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'Brazil': 55, 'China': 86, 'India': 91, 'Indonesia': 62}

In [3]:
# Handling missing keys with setdefault
import sys
import re
import os

# First non-idiomatic way
"""Build an index mapping word -> list of occurrences"""

WORD_RE = re.compile('\w+')
index = {}
zen_file = 'zen_of_python.txt'

with open(zen_file, encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group() 
            column_no = match.start() + 1
            location = (line_no, column_no)
            # This is ugly
            occurrences = index.get(word, [])
            occurrences.append(location)
            index[word] = occurrences

for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(17, 48), (18, 53)]
Although [(9, 1), (14, 1), (16, 1)]
ambiguity [(12, 16)]
and [(13, 23)]
are [(19, 12)]
aren [(8, 15)]
at [(14, 38)]
bad [(17, 50)]
be [(13, 14), (14, 27), (18, 50)]
beats [(9, 23)]
Beautiful [(1, 1)]
better [(1, 14), (2, 13), (3, 11), (4, 12), (5, 9), (6, 11), (15, 8), (16, 25)]
break [(8, 40)]
cases [(8, 9)]
complex [(3, 23)]
Complex [(4, 1)]
complicated [(4, 24)]
counts [(7, 13)]
dense [(6, 23)]
do [(13, 64), (19, 48)]
Dutch [(14, 61)]
easy [(18, 26)]
enough [(8, 30)]
Errors [(10, 1)]
explain [(17, 34), (18, 34)]
Explicit [(2, 1)]
explicitly [(11, 8)]
face [(12, 8)]
first [(14, 41)]
Flat [(5, 1)]
good [(18, 55)]
great [(19, 28)]
guess [(12, 52)]
hard [(17, 26)]
honking [(19, 20)]
idea [(17, 54), (18, 60), (19, 34)]
If [(17, 1), (18, 1)]
implementation [(17, 8), (18, 8)]
implicit [(2, 25)]
In [(12, 1)]
is [(1, 11), (2, 10), (3, 8), (4, 9), (5, 6), (6, 8), (15, 5), (16, 16), (17, 23), (18, 23)]
it [(13, 67), (17, 43), (18, 43)]
let [(19, 42)]
may [(14, 19), (18, 

The lines:
```
occurrences = index.get(word, [])
occurrences.append(location)
index[word] = occurrences
```

can be substituted with:

`index.setdefault(word, []).append(location)`


which is an equivalent way of saying:

```
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)
```

Most elegant way of handling keys which need to be initialized
would be through the collections defaultdict object.

```
index = defaultdict(list)
# and then inside the loop
my_dict[key].append(new_value)
```

`defualtdict` calls to `__getitem__` result in a fallback on `__missing__` method which in turn calls the `default_factory` function that dict was constructed with.

In [4]:
# Implement a dict which keys can be retrieved using either str or i
class StrKeyDict0(dict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

# This is done in a better style using collections module and sub
# classing from UserDict
# Otherwise we won't be able to override the builin dict __setitem__
import collections


class StrKeyDict(collections.UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)] 
    
    def __contains__(self, key):
        """This is simpler as we can assume all keys are str"""
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

There are some interesting variations of dict in standard library besides defaultdict

- `collections.OrderedDict` - maintains ordering provides popitem method

- `collections.ChainMap` - holds a list of mappings which can be searched as one:
```
import builtins
pylookup = ChainMap(locals(), globals(), vars(builtins))
```

- `collections.Counter` - mapping that holds an integer count for each key provides many useful methods like: 
    - `most_common([n])` -> list of tuples of n-most common items and their counts 
    - `subtract(iterable)` reduces the keycount in the counter by values provided in the iterable
    - It also implements all multiset operators (&, |, -, +)

In the types module you can also find wrapper class `MappingProxyType` which given mapping returns `mappingproxy` instance that is read-only but dynamic view of original mapping.